In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
train_data_dir = '/content/drive/MyDrive/Colab_Notebooks/Deep Learning/Build deep learning models with tensorflow/Image Classification/data/train'
validation_data_dir = '/content/drive/MyDrive/Colab_Notebooks/Deep Learning/Build deep learning models with tensorflow/Image Classification/data/test'



In [3]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf

BATCH_SIZE = 16

# Data augmentation and prerocessing 
print("\nLoading training data...")

training_data_generator = ImageDataGenerator(
        rescale=1./255,
        zoom_range=0.2,
        rotation_range=15,
        width_shift_range=0.05,
        height_shift_range=0.05)

training_iterator = training_data_generator.flow_from_directory(train_data_dir,class_mode='categorical',color_mode='grayscale',batch_size=BATCH_SIZE)


print("\nLoading validation data...")

#1) Create validation_data_generator, an ImageDataGenerator that just performs pixel normalization:


validation_data_generator = ImageDataGenerator(rescale = 1./255)
#2) Use validation_data_generator.flow_from_directory(...) to load the validation data from the 'data/test' folder:

validation_iterator = validation_data_generator.flow_from_directory(validation_data_dir, class_mode = 'categorical', color_mode = 'grayscale', batch_size = BATCH_SIZE)

print("\nBuilding model...")

#Creating the model, with convolutional and max pooling layers:

model = tf.keras.Sequential()
# Our input feature map is 256x256x1: 256x256 for the image pixels, and 1 for
# the color channel: grayscale
model.add(tf.keras.Input(shape=(256, 256, 1)))

# First convolution extracts 2 filters that are 5x5
# Convolution is followed by max-pooling layer with a 5x5 window
model.add(tf.keras.layers.Conv2D(2, 5, strides=3, activation="relu")) 
model.add(tf.keras.layers.MaxPooling2D(
    pool_size=(5, 5), strides=(5,5)))

# Second convolution extracts 4 filters that are 3x3
# Convolution is followed by max-pooling layer with a 2x2 window
model.add(tf.keras.layers.Conv2D(4, 3, strides=1, activation="relu")) 
model.add(tf.keras.layers.MaxPooling2D(
    pool_size=(2,2), strides=(2,2)))

# Flatten feature map to a 1-dim tensor
model.add(tf.keras.layers.Flatten())

model.add(tf.keras.layers.Dropout(0.2))

# Create output layer with a single node and softmax activation
model.add(tf.keras.layers.Dense(2,activation="softmax"))

model.summary()


print("\nCompiling model...")

#3) Compile the model with an Adam optimizer, Categorical Cross Entropy Loss, and Accuracy and AUC metrics:
opt = tf.keras.optimizers.Adam(learning_rate = 0.005)
loss = tf.keras.losses.CategoricalCrossentropy()
model.compile(
    optimizer=opt,
    loss=loss,
    metrics=[tf.keras.metrics.CategoricalAccuracy(),tf.keras.metrics.AUC()]
)

print("\nTraining model...")

#4) train and validate model for 5 epochs:

model.fit(
        training_iterator,
        steps_per_epoch=training_iterator.samples/BATCH_SIZE,
        epochs=5,
        validation_data=validation_iterator,
        validation_steps=validation_iterator.samples/BATCH_SIZE)


Loading training data...
Found 5232 images belonging to 2 classes.

Loading validation data...
Found 624 images belonging to 2 classes.

Building model...
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 84, 84, 2)         52        
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 16, 16, 2)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 14, 14, 4)         76        
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 7, 7, 4)           0         
_________________________________________________________________
flatten (Flatten)            (None, 196)               0         
_________________________________________________________________
dropout (Dropout)            (No